In [1]:
import pandas as pd
import numpy as np
import os

import transformers
from transformers import BertForSequenceClassification, AdamW, BertTokenizer, BertConfig, AutoModel
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
torch.cuda.get_device_name(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_subset = True
if train_subset:
    positive_train_comment = pd.read_csv('../data/processed/GPT/positive_train_comment.csv').dropna()
#     negative_train_comment = pd.read_csv('../data/processed/Bert-Upvote-Predictor/negative_train_comment.csv').dropna()
    train_data = []
    train_data.extend(positive_train_comment['text'].values.tolist()[0:50000])
#     train_data.extend(negative_train_comment['text'].values.tolist()[0:50000])
    train_scores = []
    train_scores.extend(positive_train_comment['score'].values.tolist()[0:50000])
#     train_scores.extend(negative_train_comment['score'].values.tolist()[0:50000])
else:
    positive_train_comment = pd.read_csv('../data/processed/GPT/positive_train_comment.csv').dropna()
#     negative_train_comment = pd.read_csv('../data/processed/Bert-Upvote-Predictor/negative_train_comment.csv').dropna()
    train_data = []
    train_data.extend(positive_train_comment['text'].values.tolist())
#     train_data.extend(negative_train_comment['text'].values.tolist())
    train_scores = []
    train_scores.extend(positive_train_comment['score'].values.tolist())
#     train_scores.extend(negative_train_comment['score'].values.tolist())
assert(len(train_data) == len(train_scores))
print(train_data[0:5])
print(train_scores[0:5])

['Read your name as limptits.', 'illness', '"Well, he either died on the toilet, or brought his phone in..."', 'how old are you and how long do you think it took for this to build up?', ' When an electrical storm hit Funkyy\'s television on one fateful night, his apathy transformed into...  telepathy "I\'m hearing voices in my head! I think I\'m going crazy!"  His talent earned him fame and riches... "I\'m famous and rich!"  But he was about to find out...  "I\'m about to find out!"   ...Success was not quite what he expected "Make love to me, Funkyy!"  "No me, make love to me!" "Funkkkkyyyyyyyy!"  "Funkky, the President requests you leave congress. You are just too lazy"  "Duuuuuude" The Feel-Good comedy of the YearEDDIE MURPHY  MICHAEL CAINE  JESSICA ALBA  amp; introducing,  FUNKYY   Out Now, on Blu-Ray and DVD   ★★★☆☆ - Time Out Magazine  "Meh!!" Sometimes you just gotta slack off!  Successful Slacking!! ']
[236, 73, 107, 68, 141]


In [4]:
positive_valid_comment = pd.read_csv('../data/processed/GPT/positive_valid_comment.csv').dropna()
# negative_valid_comment = pd.read_csv('../data/processed/Bert-Upvote-Predictor/negative_valid_comment.csv').dropna()
valid_data = []
valid_data.extend(positive_valid_comment['text'].values.tolist())
# valid_data.extend(negative_valid_comment['text'].values.tolist())
valid_scores = []
valid_scores.extend(positive_valid_comment['score'].values.tolist())
# valid_scores.extend(negative_valid_comment['score'].values.tolist())
assert(len(valid_data) == len(valid_scores))
print(valid_data[0:5])
print(valid_scores[0:5])

['Maybe she was scared of reacting positively to meeting you again.', 'This text is Bel Air free and thus safe to read.', "That's the worst analogy for sex I've ever heard.", 'The marketability of breast cancer is really taking away from funding that could go to say..colon cancer, a cancer which is much more preventable and afflicts many more people.  Lets have a colon cancer awareness month where we where brown ribbons, or green ribbons for the leafy veggie eaters.', 'A programmer has a rare disease, and has part of his digestive tract removed to cure it. He is never truly able to live with himself afterward because he is missing a semicolon.']
[71, 100, 391, 78, 397]


In [5]:
# max_length = 0
# max_sentence = None

# def tokenize(sentence):
#     global max_length
#     global max_sentence
#     sentence_list = sentence.split()
#     ret = []
#     for word in sentence_list:
#         ret.extend(tokenizer.tokenize(word))
#     if len(ret) > max_length:
#         max_sentence = sentence
#         max_length = len(ret)
#     return ret

In [6]:
# def transform(sentences):
#     tokenized_sentences = [tokenize(sentence) for sentence in sentences]
#     print(len(tokenized_sentences))
#     print(max(list(map(lambda x:len(x), tokenized_sentences))))

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)

# encode text
tokenized_train = tokenizer.batch_encode_plus(train_data, max_length=512, pad_to_max_length=True, truncation=True)
tokenized_val = tokenizer.batch_encode_plus(valid_data, max_length=512, pad_to_max_length=True, truncation=True)


/scratch/dj1057/nlp/code/transformers/src/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [8]:
print(len(tokenized_train['input_ids'][0]))
tokenized_train['input_ids'][0]

512


[101,
 100,
 2115,
 2171,
 2004,
 14401,
 3775,
 3215,
 1012,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [9]:
train_seq = torch.tensor(tokenized_train['input_ids']).to(device)
train_mask = torch.tensor(tokenized_train['attention_mask']).to(device)
train_y = torch.tensor(train_scores).to(device)

val_seq = torch.tensor(tokenized_val['input_ids']).to(device)
val_mask = torch.tensor(tokenized_val['attention_mask']).to(device)
val_y = torch.tensor(valid_scores).to(device)

In [10]:
train_seq.shape

torch.Size([50000, 512])

In [11]:
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [12]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 1,
    return_dict=False,
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda();
optimizer = AdamW(model.parameters(),  lr = 2e-5, eps = 1e-8 ) 
epochs = 10

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
def train(model):
    model.train()
    
    total_loss= 0
    print('begin train')
    for step,batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
        sent_id, mask, labels = batch
        model.zero_grad()        
        sent_id = sent_id.to(device)
        mask = mask.to(device)
        labels = labels.to(device).float()
        loss, logits = model(sent_id, token_type_ids=None, attention_mask=mask, labels=labels)
        total_loss = total_loss + loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
    avg_loss = total_loss / len(train_dataloader)
    
    return avg_loss

In [14]:
def evaluate(model):
    print("\nEvaluating...")
    model.eval()
    total_loss= 0
    
    for step,batch in enumerate(val_dataloader):
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
        sent_id, mask, labels = batch
        sent_id = sent_id.to(device)
        mask = mask.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            loss, logits = model(sent_id, 
                             token_type_ids=None, 
                             attention_mask=mask, 
                             labels=labels)
            total_loss = total_loss + loss.item()
    avg_loss = total_loss / len(val_dataloader) 
    return avg_loss

In [ ]:
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss= train(model)
    
    #evaluate model
    print("evaluate")
    valid_loss= evaluate(model)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
begin train
  Batch    50  of  1,563.
  Batch   100  of  1,563.
  Batch   150  of  1,563.
  Batch   200  of  1,563.
  Batch   250  of  1,563.
  Batch   300  of  1,563.
  Batch   350  of  1,563.
  Batch   400  of  1,563.
  Batch   450  of  1,563.
  Batch   500  of  1,563.
  Batch   550  of  1,563.
  Batch   600  of  1,563.
  Batch   650  of  1,563.
  Batch   700  of  1,563.
  Batch   750  of  1,563.
  Batch   800  of  1,563.
  Batch   850  of  1,563.
